In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
import math
from sklearn.metrics import mean_absolute_error


In [2]:
dataOriginal=pd.read_csv('./ml-100k/u.data',sep=r'\t',names=['user id','item id', 'rating', 'timestamp'])
filmesId=pd.DataFrame((set(dataOriginal['item id'])),columns=['Filmes'])

itemData=pd.read_csv('./ml-100k/u.item',sep=r'|',names=['movie_id ', 'movie title' , 'release date' ,' video release date' ,
              'IMDb URL' ,' unknown' , 'Action' , 'Adventure' , 'Animation' ,
             ' Childrens ', 'Comedy' , 'Crime' , 'Documentary' , 'Drama' , 'Fantasy' ,
              'Film-Noir' , 'Horror' , 'Musical' , 'Mystery' , 'Romance' , 'Sci-Fi' ,
              'Thriller' , 'War' , 'Western' ], encoding='latin-1')
userData=pd.read_csv('./ml-100k/u.user',sep='|',names=['user id','age','Genero','Ocupacao','ZipCode'])

C:\Users\User\AppData\Local\Temp\ipykernel_8244\1420820392.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  dataOriginal=pd.read_csv('./ml-100k/u.data',sep=r'\t',names=['user id','item id', 'rating', 'timestamp'])


In [3]:


def cossine_similarity(u, v,tabelaSimilaridade):
    u=u.reset_index()
    v=v.reset_index()
    itemsComuns = set(u['item id']).intersection(set(v['item id']))
    id_u=u['user id'][0]
    id_v = v['user id'][0]
    u = u[u['item id'].isin(itemsComuns)].sort_values(by='item id')['rating']
    v = v[v['item id'].isin(itemsComuns)].sort_values(by='item id')['rating']
    result = np.dot(u, v)/(np.linalg.norm(u)*np.linalg.norm(v))
    tabelaSimilaridade[id_u][id_v]=result
    tabelaSimilaridade[id_v][id_u]=result
   
    # print(u,v)
    return result


def user_based(user_id, item_id,data,tabelaSimilaridade):
    Nui = userData[userData['user id'].isin(
        data[data['item id'] == item_id]['user id'])]['user id']
    # print(""+str(user_id)+" "+str(item_id))
    # print(len(Nui))
    if(len(Nui)==0):
        return None
    # parcial1 = sum(Nui.apply(lambda x:
    #                          ((cossine_similarity(data[data['user id'] == user_id],
    #                                              data[data['user id']
    #                                                   == x],tabelaSimilaridade
    #                                              )if math.isnan(tabelaSimilaridade[user_id][x])else  tabelaSimilaridade[user_id][x])*(data[(
    #                                             data['user id'] == x) & (data['item id'] == item_id)
    #                                              ]['rating'].reset_index(drop=True)), axis=1

    #                           ).rename(columns={0: 'rating'})['rating'])
    parcial1 = sum(Nui.apply(lambda x:
                             ((cossine_similarity(data[data['user id'] == user_id],
                                                 data[data['user id']
                                                      == x],tabelaSimilaridade
                                                  )if math.isnan(tabelaSimilaridade[user_id][x])else tabelaSimilaridade[user_id][x])
                                                  *data[(
                                                     data['user id'] == x) & (data['item id'] == item_id)
                             ]['rating'].reset_index(drop=True))

                             ).rename(columns={0: 'rating'})['rating'])
                             
    # parcial1 = sum(Nui.apply(lambda x:
    #                          (
    #                             print("foi")
    #                              if math.isnan(tabelaSimilaridade[user_id][x['user id']]) else print("Nao foi")
    #                              )
    #                          ))
    parcial2 = sum(Nui.apply(lambda x:
                             (abs(cossine_similarity(data[data['user id'] == user_id],
                                                     data[data['user id']
                                                          == x],
                                                          tabelaSimilaridade
                                                     )) if math.isnan( tabelaSimilaridade[user_id][x])  else abs(tabelaSimilaridade[user_id][x]) )
                             ))
    return parcial1/parcial2


In [4]:

train,test=train_test_split(dataOriginal.drop('timestamp',axis=1), test_size=0.1, random_state=42)
x_test=test.drop('rating',axis=1)
y_test=test['rating']
x_test

,user id,item id
75721,877,381
80184,815,602
19864,94,431
76699,416,875
92991,500,182
...,...,...
5002,222,164
30151,465,855
93194,249,93
73199,807,174


In [5]:
similaridade = pd.DataFrame(index=userData['user id'],columns=userData['user id'])
for id in userData['user id']:
    similaridade[id][id]=1
similaridade


user id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
user id,,,,,,,,,,,,,,,,,,,,,
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN
941,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN


In [7]:
y_test_predict= x_test.apply(lambda x:
           user_based(x['user id'],x['item id'],train,similaridade) ,axis=1)
y_test_predict

C:\Users\User\AppData\Local\Temp\ipykernel_8244\2896778804.py:9: RuntimeWarning: invalid value encountered in scalar divide
  result = np.dot(u, v)/(np.linalg.norm(u)*np.linalg.norm(v))
C:\Users\User\AppData\Local\Temp\ipykernel_8244\2896778804.py:9: RuntimeWarning: invalid value encountered in scalar divide
  result = np.dot(u, v)/(np.linalg.norm(u)*np.linalg.norm(v))
C:\Users\User\AppData\Local\Temp\ipykernel_8244\2896778804.py:9: RuntimeWarning: invalid value encountered in scalar divide
  result = np.dot(u, v)/(np.linalg.norm(u)*np.linalg.norm(v))
C:\Users\User\AppData\Local\Temp\ipykernel_8244\2896778804.py:9: RuntimeWarning: invalid value encountered in scalar divide
  result = np.dot(u, v)/(np.linalg.norm(u)*np.linalg.norm(v))
C:\Users\User\AppData\Local\Temp\ipykernel_8244\2896778804.py:9: RuntimeWarning: invalid value encountered in scalar divide
  result = np.dot(u, v)/(np.linalg.norm(u)*np.linalg.norm(v))
C:\Users\User\AppData\Local\Temp\ipykernel_8244\2896778804.py:9: Runti

75721    3.599528
80184    3.700872
19864    3.563567
76699    2.759552
92991    3.927348
           ...   
5002     3.606332
30151    3.851925
93194    3.710477
73199    4.253536
74479         NaN
Length: 10000, dtype: float64

In [15]:
y_predict=y_test_predict.replace(np.nan,0)
y_predict[y_predict==0]

84004    0.0
60767    0.0
41331    0.0
28533    0.0
61769    0.0
        ... 
86640    0.0
27466    0.0
98513    0.0
48808    0.0
74479    0.0
Length: 1187, dtype: float64

In [13]:
y_test

75721    4
80184    3
19864    4
76699    2
92991    2
        ..
5002     4
30151    4
93194    4
73199    5
74479    3
Name: rating, Length: 10000, dtype: int64

In [14]:
similaridade

user id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
user id,,,,,,,,,,,,,,,,,,,,,
1,1,0.961773,0.823586,0.868586,0.932354,0.952203,0.943472,0.98152,0.969117,0.967905,...,0.934217,0.855561,0.964998,0.886225,0.887381,0.946955,0.968153,0.965431,0.894197,0.928138
2,0.961773,1,0.952399,0.946544,NaN,0.958162,0.9674,NaN,0.942524,0.987842,...,0.943838,0.958885,0.976242,0.971183,0.933087,0.985318,0.940248,0.936329,0.936975,NaN
3,0.823586,0.952399,1,0.92321,NaN,NaN,0.850045,0.864791,1.0,0.93091,...,0.980581,1.0,NaN,0.852056,0.8412,NaN,0.854179,1.0,0.943322,1.0
4,0.868586,0.946544,0.92321,1,1.0,0.930663,0.865838,0.972529,1.0,0.993884,...,0.970143,NaN,0.976187,NaN,0.985721,NaN,0.96266,1.0,0.973123,0.990148
5,0.932354,NaN,NaN,1.0,1,0.927557,0.910627,0.946131,NaN,0.940531,...,0.914503,NaN,0.933026,0.976315,0.885686,0.944787,0.915168,0.972014,0.946685,0.922994
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.946955,0.985318,NaN,NaN,0.944787,0.928541,0.958731,NaN,NaN,1.0,...,0.947331,0.963552,0.975398,0.907877,0.936319,1,0.984732,0.952956,NaN,0.95409
940,0.968153,0.940248,0.854179,0.96266,0.915168,0.946265,0.935733,0.957312,NaN,0.966163,...,0.930322,0.964252,0.905238,0.828795,0.969964,0.984732,1,0.986797,0.954643,0.96301
941,0.965431,0.936329,1.0,1.0,0.972014,0.948504,0.901397,0.906361,NaN,0.995228,...,0.948683,0.953817,0.966708,0.94554,0.936382,0.952956,0.986797,1,0.978232,NaN
